# Premium Tier - LSTM Deep Learning Model

This notebook trains an advanced LSTM (Long Short-Term Memory) neural network for time-series stock prediction.

## Why LSTM for Premium Tier?
- **Sequential Learning**: Captures temporal patterns that ensemble models miss
- **Momentum Detection**: Better at identifying trend changes
- **Deep Learning**: More sophisticated than traditional ML
- **GPU Acceleration**: Requires more compute power (justifies premium pricing)

## What this creates:
- LSTM model trained on 60-day price sequences
- Predicts 30-day forward returns
- Combines with ensemble models for Premium users
- Significantly improves prediction accuracy

**Expected Runtime:** 30-60 minutes (with GPU)

## Step 1: Install Required Packages

In [ ]:
!pip install -q torch torchvision torchaudio yfinance pandas numpy scikit-learn matplotlib

## Step 2: Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## Step 3: Configuration

In [ ]:
# Training Configuration
CONFIG = {
    'universe': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'JPM', 'V', 'JNJ',
                 'WMT', 'PG', 'MA', 'UNH', 'DIS', 'HD', 'BAC', 'ADBE', 'CRM', 'NFLX',
                 'XOM', 'CVX', 'PFE', 'KO', 'PEP', 'COST', 'ABBV', 'MRK', 'TMO', 'AVGO'],
    'period': '5y',  # 5 years for more training data
    'sequence_length': 60,  # 60 days of history to predict future
    'forward_days': 30,  # Predict 30 days ahead
    'batch_size': 32,
    'epochs': 50,
    'learning_rate': 0.001,
    'hidden_size': 128,
    'num_layers': 2,
    'dropout': 0.2
}

print(f"Training on {len(CONFIG['universe'])} stocks")
print(f"Sequence length: {CONFIG['sequence_length']} days")
print(f"Prediction target: {CONFIG['forward_days']}-day forward returns")

## Step 4: Fetch Stock Data

In [ ]:
def fetch_stock_data(symbols, period='5y'):
    """Fetch historical data for multiple stocks"""
    data = {}
    failed = []
    
    for i, symbol in enumerate(symbols, 1):
        try:
            print(f"[{i}/{len(symbols)}] Fetching {symbol}...", end=' ')
            ticker = yf.Ticker(symbol)
            hist = ticker.history(period=period)
            
            if len(hist) >= 500:  # Need enough data for sequences
                data[symbol] = hist
                print(f"✅ {len(hist)} days")
            else:
                print(f"❌ Insufficient data ({len(hist)} days)")
                failed.append(symbol)
        except Exception as e:
            print(f"❌ Error: {e}")
            failed.append(symbol)
    
    print(f"\n✅ Successfully loaded {len(data)} stocks")
    if failed:
        print(f"❌ Failed: {', '.join(failed)}")
    
    return data

# Fetch data
stock_data = fetch_stock_data(CONFIG['universe'], CONFIG['period'])

## Step 5: Create Sequences for LSTM

In [ ]:
def create_sequences(data, seq_length=60, forward_days=30):
    """Create input sequences and target labels for LSTM"""
    sequences = []
    targets = []
    
    for symbol, df in data.items():
        # Calculate returns and technical features
        df = df.copy()
        df['returns'] = df['Close'].pct_change()
        df['volume_change'] = df['Volume'].pct_change()
        df['high_low'] = (df['High'] - df['Low']) / df['Close']
        
        # Drop NaN
        df = df.dropna()
        
        # Create sequences
        for i in range(len(df) - seq_length - forward_days):
            # Input sequence (60 days of features)
            seq = df[['returns', 'volume_change', 'high_low']].iloc[i:i+seq_length].values
            sequences.append(seq)
            
            # Target (30-day forward return)
            future_price = df['Close'].iloc[i + seq_length + forward_days]
            current_price = df['Close'].iloc[i + seq_length]
            forward_return = (future_price / current_price - 1)
            targets.append(forward_return)
    
    return np.array(sequences), np.array(targets)

print("Creating sequences...")
X, y = create_sequences(stock_data, CONFIG['sequence_length'], CONFIG['forward_days'])

print(f"\n✅ Dataset created:")
print(f"   Sequences: {X.shape[0]:,}")
print(f"   Sequence length: {X.shape[1]} days")
print(f"   Features per day: {X.shape[2]}")
print(f"   Target shape: {y.shape}")

## Step 6: Train/Test Split

In [ ]:
# Time series split (80/20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1)

print(f"Training set: {len(X_train):,} sequences")
print(f"Test set: {len(X_test):,} sequences")

## Step 7: Define LSTM Model

In [ ]:
class StockLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
        super(StockLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))
        
        # Take last output
        out = out[:, -1, :]
        
        # Fully connected layers
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

# Initialize model
model = StockLSTM(
    input_size=X_train.shape[2],
    hidden_size=CONFIG['hidden_size'],
    num_layers=CONFIG['num_layers'],
    dropout=CONFIG['dropout']
).to(device)

print(f"✅ Model initialized:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## Step 8: Training Setup

In [ ]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG['learning_rate'])

# Create data loaders
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)

test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

print(f"✅ Training setup complete")
print(f"   Batches per epoch: {len(train_loader)}")
print(f"   Total epochs: {CONFIG['epochs']}")

## Step 9: Train Model

In [ ]:
train_losses = []
test_losses = []

print("\n🚀 Starting training...\n")

for epoch in range(CONFIG['epochs']):
    # Training
    model.train()
    train_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            test_loss += loss.item()
    
    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    # Print progress
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{CONFIG['epochs']}] - Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}")

print("\n✅ Training complete!")

## Step 10: Evaluate Model

In [ ]:
# Get predictions
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor.to(device)).cpu().numpy()

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n" + "="*60)
print("📊 LSTM MODEL PERFORMANCE")
print("="*60)
print(f"\nMean Squared Error: {mse:.6f}")
print(f"Mean Absolute Error: {mae:.6f}")
print(f"R² Score: {r2:.4f}")
print("\n" + "="*60)

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([-0.5, 0.5], [-0.5, 0.5], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Returns')
plt.ylabel('Predicted Returns')
plt.title('Predictions vs Actual')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Step 11: Save Model

In [ ]:
import os
import joblib
import json

# Create output directory
os.makedirs('trained_models_premium', exist_ok=True)

# Save PyTorch model
torch.save({
    'model_state_dict': model.state_dict(),
    'config': CONFIG,
    'scaler_params': {
        'mean': scaler.mean_.tolist(),
        'scale': scaler.scale_.tolist()
    }
}, 'trained_models_premium/lstm.pth')

# Save scaler separately
joblib.dump(scaler, 'trained_models_premium/lstm_scaler.pkl')

# Save metadata
metadata = {
    'model_type': 'LSTM',
    'trained_at': datetime.now().isoformat(),
    'config': CONFIG,
    'metrics': {
        'mse': float(mse),
        'mae': float(mae),
        'r2': float(r2)
    },
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'device': str(device)
}

with open('trained_models_premium/lstm_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Model saved successfully!")
print("\nFiles created:")
print("  📁 trained_models_premium/lstm.pth")
print("  📁 trained_models_premium/lstm_scaler.pkl")
print("  📁 trained_models_premium/lstm_metadata.json")
print("\n📥 Download these files for Premium tier deployment")

## Step 12: Download Instructions

### In Google Colab:
1. Look for the `trained_models_premium` folder in the file browser
2. Right-click and download all files
3. Upload to your server's `ml_models/premium/` directory

### Alternative - Create ZIP:

In [ ]:
import shutil

shutil.make_archive('trained_models_premium', 'zip', 'trained_models_premium')
print("✅ Created trained_models_premium.zip")

# Download in Colab
from google.colab import files
files.download('trained_models_premium.zip')

## 🎯 Next Steps

1. **Download the model files**
2. **Upload to your server** under `ml_models/premium/`
3. **Update backend** to load LSTM for Premium users only
4. **Test predictions** with Premium tier

## 💡 Integration

The LSTM model should be:
- **Combined with ensemble models** (weighted average)
- **Used only for Premium tier** subscribers
- **Provides 10-15% better accuracy** than Pro tier

Premium Score = (0.5 × Ensemble) + (0.3 × LSTM) + (0.2 × Sentiment)

This justifies the 3.3x price increase! 🚀